In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace

scala_version = '2.13'
spark_version = '3.5.5'
packages = [ f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}','org.apache.kafka:kafka-clients:2.10.0']
spark = SparkSession.builder.master('local')\
.appName("kafka-example").config("spark.jars.packages", ",".join(packages))\
.config("spark.memory.offHeap.enabled","true")\
.config("spark.memory.offHeap.size","10g")\
.config("spark.driver.memory", "16g")\
.getOrCreate()

In [2]:
topic_name = 'VideoStreaming'
kafka_server = 'localhost:9092'

streamRawDf = spark.readStream.format("kafka")\
    .option("kafka.bootstrap.servers", kafka_server)\
    .option("subscribe", topic_name)\
    .load()

streamDF = streamRawDf.select(col('topic'),col('offset'),col('value').cast('string'), col('partition'))

In [3]:
stream_writer1 = (streamDF.writeStream.queryName("RandomNumberQuery").trigger(processingTime="5 seconds").outputMode("append").format("memory"))

query1 = stream_writer1.start()    


In [ ]:
from time import sleep
from IPython.display import display, clear_output
for x in range(0, 100):
    try:
        print("Showing live view refreshed every 5 seconds")
        print(f"Seconds passed: {x*5}")
        result1 = spark.sql(f"SELECT * FROM {query1.name}")
        display(result1.toPandas())

        sleep(3)
        clear_output(wait=True)
    except KeyboardInterrupt:
        print("break")
        break
print("Live view ended...")


Showing live view refreshed every 5 seconds
Seconds passed: 100


,topic,offset,value,partition
0,VideoStreaming,1568,���� JFIF      �� C ...,2
1,VideoStreaming,1569,���� JFIF      �� C ...,2
2,VideoStreaming,1570,���� JFIF      �� C ...,2
3,VideoStreaming,1571,���� JFIF      �� C ...,2
4,VideoStreaming,1572,���� JFIF      �� C ...,2
...,...,...,...,...
1209,VideoStreaming,2927,���� JFIF      �� C ...,1
1210,VideoStreaming,2928,���� JFIF      �� C ...,1
1211,VideoStreaming,2929,���� JFIF      �� C ...,1
1212,VideoStreaming,2930,���� JFIF      �� C ...,1


break
Live view ended...
